# Data Cleaning and Transformation

In the last chapter, we created functions to download and save the raw data. In this chapter, we take steps to find and clean bad data, and transform it to a structure that is suitable for modeling. We begin by reading in the raw local data with the `run` function.

### Important note on importing from solutions

Because this is a new chapter, you will need to import functions from the solutions.py file that were defined in the previous chapter. Continue to comment out the import statements that appear after an exercise so that you can practice them. Alternatively, you can put all of your solutions in a solutions2.py file and change the import statements to import your functions.

In [ ]:
import pandas as pd
from solutions import run
data = run()
data['world_cases'].head(3)

In [ ]:
data['usa_cases'].head(3)

## Selecting the correct columns

Take a look at the world and USA DataFrames above and you'll notice a difference in the names and number of columns. The following exercise describes how to select the columns.

### Exercise 6

<span style="color:green; font-size:16px">Write a function that accepts a single DataFrame and selects the `"Country/Region"` column for the world DataFrames, `"Province_State"` column for the USA DataFrames, and all the date columns for both. Return a DataFrame with just those columns. Assume that the column names always remain the same for each dataset.</span>

In [ ]:
def select_columns(df):
    """
    Selects the Country/Region column for world DataFrames and
    Province_State for USA
    
    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    df : DataFrame
    
    """
    pass

Let's use this function to select the columns and output from both the world and USA DataFrames.

In [ ]:
from solutions import select_columns
select_columns(data['world_cases']).head(3)

In [ ]:
select_columns(data['usa_cases']).head(3)

### Updating the `run` function

After each step in this chapter, we'll update our `run` function to pass each DataFrame through the newly created function. Each `run` function will be uniquely labeled with an ending integer.

### Exercise 7

<span style="color:green; font-size:16px">Update the `run` function to include the above step.</span>

In [ ]:
def run2():
    """
    Run all cleaning and transformation steps
    
    Returns
    -------
    Dictionary of DataFrames
    """
    pass

In [ ]:
from solutions import run2
data = run2()
data['usa_cases'].head(3)

## Updating area names 

In both groups of data, there are a few area names that can be updated so that they use a more common name. There are three cruise ships, which we will replace with the string "Cruise Ship". Also, since the United States has its own summary table, we can drop it from the world DataFrames.

### Exercise 8

<span style="color:green; font-size:16px">Write a function that uses the DataFrame `replace` method to replace the names in the first column with the provided dictionary below. Drop all rows from the "US" from the world DataFrame.</span>

In [ ]:
REPLACE_AREA = {
    "Korea, South": "South Korea",
    "Taiwan*": "Taiwan",
    "Burma": "Myanmar",
    "Holy See": "Vatican City",
    "Diamond Princess": "Cruise Ship",
    "Grand Princess": "Cruise Ship",
    "MS Zaandam": "Cruise Ship"
}

def update_areas(df):
    """
    Replace a few of the area names using the REPLACE_AREA dictionary.
    
    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    df : DataFrame
    """
    pass

### Exercise 9

<span style="color:green; font-size:16px">Update the `run` function to include the above step.</span>

In [ ]:
def run3():
    """
    Run all cleaning and transformation steps
    
    Returns
    -------
    Dictionary of DataFrames
    """
    pass

We verify our function by searching for the cruise ships.

In [ ]:
from solutions import run3
data = run3()
data['usa_cases'].query("Province_State == 'Cruise Ship'")

## Aggregate repeating areas

In each DataFrame, many areas repeat multiple times as the raw data tracked deaths/cases by the province/state/county level. We desire a single row for each unique area. Complete the exercise below to get the desired result.

### Exercise 10

<span style="color:green; font-size:16px">Write a function that accepts a single DataFrame, groups by the area column (first column in each DataFrame), and sums up all the date columns.</span>

In [ ]:
def group_area(df):
    """
    Gets a single total for each area
    
    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    df : DataFrame
    """
    pass

### Exercise 11

<span style="color:green; font-size:16px">Update the `run` function to include the above step.</span>

In [ ]:
def run4():
    """
    Run all cleaning and transformation steps
    
    Returns
    -------
    Dictionary of DataFrames
    """
    pass

In [ ]:
from solutions import run4
data = run4()
data['usa_cases'].head(3)

## Transposing the data to time series

We have time series data (a sequence of data over time), but it's not in the customary format where date is along the vertical axis. Complete the following exercise to convert it to a more common format for time series.

### Exercise 12

<span style="color:green; font-size:16px">Write a function that accepts a single DataFrame and transposes it so that the current date columns become the index. Make sure to convert the dates to a datetime data type, since they are strings now.</span>

In [ ]:
def transpose_to_ts(df):
    """
    Transposes the DataFrame and converts the index to datetime

    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    df : DataFrame
    """
    pass

### Exercise 13

<span style="color:green; font-size:16px">Update the `run` function to include the above step.</span>

In [ ]:
def run5():
    """
    Run all cleaning and transformation steps
    
    Returns
    -------
    Dictionary of DataFrames
    """
    pass

In [ ]:
from solutions import run5
data = run5()
data['usa_cases'].tail(3)

## Finding and handling bad data

In this section, we will search for bad data and then come up with a solution for handling it. Our DataFrames contain the cumulative count of deaths and cases at each date. These values should never decrease. In order to verify that the values never decrease, we can test whether each day's value is at least as large as all the values preceding it. To do this, we call the `cummax` method which returns the cumulative maximum of each column up to each date. We then compare each value with this cumulative maximum. We'll work with just the world deaths DataFrame for now.

In [ ]:
world_deaths = data['world_deaths']
bad_data = world_deaths < world_deaths.cummax()
bad_data.tail(3)

If any of these values are `True`, then we've found bad data. Let's sum each column and sort the results to see which columns have the most bad data.

In [ ]:
bad_data.sum().sort_values(ascending=False).head(10)

Let's locate the bad data for Spain, and see if we can find out what's happening.

In [ ]:
spain_bad = bad_data['Spain']
spain_bad[spain_bad].head()

Let's inspect a small subset of the data around the first date of bad data.

In [ ]:
world_deaths.loc['2020-05-21':'2020-05-26', 'Spain']

A drop of nearly 2,000 deaths appears on May 25th. Let's make a plot of Spain's total deaths beginning from the beginning of May to get a better picture of what is happening.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dashboard.mplstyle')
world_deaths.loc['2020-05-01':'2020-06-01', 'Spain'].plot();

It appears that almost no new deaths are reported after the sudden decrease on May 25th, until a huge increase in the latter half of June, followed again by a period of very few deaths. Various other data aggregators have reported similar issues with Spain's data.

We'll provide a simple solution so that all dates have a value greater than or equal to the prior day's values. In order to have make this replacement, we'll change all the values for dates below the current maximum to missing values with the `mask` method. First, we create a boolean mask, a Series of booleans with the same length as the original Series that meet some criteria.

In [ ]:
spain = world_deaths['Spain']
mask = spain < spain.cummax()
mask.tail()

We pass this Series to the `mask` method to "mask" them - cover them up and replace them with missing values. We show the first 10 dates where the data is now missing.

In [ ]:
spain_masked = spain.mask(mask)
spain_masked[spain_masked.isna()].head(10)

We can then linearly interpolate the missing values with the `interpolate` method and plot the updated data.

In [ ]:
spain_masked.interpolate().plot();

This "fixes" the data such that each value is always at least as large as the preceding value. In this particular example, this simple fix doesn't seem to connect the points in a way pleasing to the eye. A better estimation might linearly interpolate from the middle of May to the middle of July. 

Instead of developing a more complex method to fix bad data, we'll use this simple method and complete a process called **smoothing** later on, which will really help out the model handle these uneven jumps in the data.

### Fixing all bad data

Let's fix all of the bad data in our DataFrame with the same logic from above, rounding the totals to whole numbers.

In [ ]:
mask = world_deaths < world_deaths.cummax()
world_deaths_fixed = world_deaths.mask(mask).interpolate().round(0).astype('int64')
world_deaths_fixed.tail(3)

Let's verify that all values are at least as large as the previous day's value.

In [ ]:
mask = world_deaths_fixed < world_deaths_fixed.cummax()
mask.sum().sum()

### Exercise 14

<span style="color:green; font-size:16px">Write a function that accepts a single DataFrame and fixes all the bad data.</span>

In [ ]:
def fix_bad_data(df):
    """
    Replaces all days for each country where the value of
    deaths/cases is lower than the current maximum
    
    Parameters
    ----------
    df : DataFrame
    
    Returns
    -------
    DataFrame
    """
    pass

### Exercise 15

<span style="color:green; font-size:16px">Update the `run` function to include the above step.</span>

In [ ]:
def run6():
    """
    Run all cleaning and transformation steps
    
    Returns
    -------
    Dictionary of DataFrames
    """
    pass

Verify that this last step works:

In [ ]:
from solutions import run6
data = run6()
data['world_cases'].head(3)

## Data preparation complete

These steps complete the data preparation process. Let's use one of our previous functions to write this prepared data to the `data/prepared` folder.

In [ ]:
from solutions import write_data
write_data(data, 'data/prepared', index=True, index_label='date')

## Encapsulate all steps into a single class

All of the steps in the last two chapters may be encapsulated into a single class.

### Exercise 16

<span style="color:green; font-size:16px">Write a class that has a method for each of the steps from the last two chapter. Add a `run` method that runs all of the steps and returns the dictionary of DataFrames. Initialize the object with the `download_new` boolean, which allows the user to decide whether to download new data from the online repository or read in the local data. Check the `prepare.py` file for the solution.</span>

In [ ]:
class PrepareData:
    def __init__(self, download_new=True):
        self.download_new = download_new

To check your work run the following code:

In [ ]:
from prepare import PrepareData
prepare_data = PrepareData()
data = prepare_data.run()
data['world_deaths'].head()